In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.kernel_approximation import RBFSampler
from sklearn.linear_model import RidgeClassifier

In [ ]:
# Importing Important Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
df = pd.read_csv('Kerala_with_lag.csv', parse_dates = ["Date"])
df.head()

,Unnamed: 0,Date,Rain,T_max,T_min,Tourism,Travel_index,Travel,Bus & Rail,Tourist Destinations,...,Cluster_ID,Tourist Arrivals,date,dayofweek,quarter,month,year,dayofyear,dayofmonth,weekofyear
0,0,2010-01-06,1.424957,0.665612,0.701527,2.617251,1.956571,1.948925,0.631997,2.130383,...,3,26738.0,0,2,1,1,2010,6,6,1
1,1,2010-01-07,1.925312,-0.359186,0.348964,2.617251,2.127279,2.119442,1.348376,1.638271,...,4,40931.0,1,3,1,1,2010,7,7,1
2,2,2010-01-08,0.622243,-0.895985,0.569315,2.060617,2.297986,2.289959,2.064754,2.212402,...,3,56230.0,2,4,1,1,2010,8,8,1
3,3,2010-01-09,0.068229,-0.950885,0.619682,3.034726,2.724755,2.716253,1.437923,3.114608,...,5,44673.0,3,5,1,1,2010,9,9,1
4,4,2010-01-10,0.639319,-0.944785,0.355259,2.756409,2.468694,2.460477,1.437923,2.212402,...,3,59414.0,4,6,1,1,2010,10,10,1


In [ ]:
df_test = pd.read_csv('shimla_with_lag.csv', parse_dates = ["Date"])
df_test.head()

,Date,Rain,T_max,T_min,Tourism,Travel_index,Travel,Bus & Rail,Tourist Destinations,Travel Agencies,value_lag_1,value_lag_2,value_lag_3,value_lag_4,value_lag_5,Tourist Arrivals
0,2010-06-01,-0.397005,1.006507,1.006507,4.023852,3.992883,2.372893,1.821047,3.183420,0,390553.0,395110.0,198760.0,174609.0,183556.0,432559.0
1,2010-07-01,-0.275566,0.937469,0.937469,1.835582,1.370189,0.642150,0.787077,1.083396,0,432559.0,390553.0,395110.0,198760.0,174609.0,197096.0
2,2010-08-01,0.765266,0.943605,0.943605,1.085318,0.878434,0.019083,-0.053024,0.411389,2,197096.0,432559.0,390553.0,395110.0,198760.0,222870.0
3,2010-09-01,0.373287,1.081681,1.081681,1.022796,1.042353,0.434461,0.334715,0.831394,4,222870.0,197096.0,432559.0,390553.0,395110.0,207944.0
4,2010-10-01,-0.397005,0.379028,0.379028,1.022796,2.681536,0.365231,0.528584,0.495390,0,207944.0,222870.0,197096.0,432559.0,390553.0,154716.0


In [ ]:
# df["Recreation"] = df["Recreation"].astype(int)
# df["Car & Taxi Rental"] = df["Car & Taxi Rental"].astype(int)

In [ ]:
def create_features(df, label=None):
    """
    Creates time series features from datetime index
    """
    df['date'] = df.index
    df['dayofweek'] = df['Date'].dt.dayofweek
    df['quarter'] = df['Date'].dt.quarter
    df['month'] = df['Date'].dt.month
    df['year'] = df['Date'].dt.year
    df['dayofyear'] = df['Date'].dt.dayofyear
    df['dayofmonth'] = df['Date'].dt.day
    df['weekofyear'] = df['Date'].dt.weekofyear

    X = df[['quarter','month','year','Rain', 'T_max', 'T_min', 'Tourism', 'Travel_index', 'Travel', 'Bus & Rail', 'Tourist Destinations', 'value_lag_1', 'value_lag_2', 'value_lag_3', 'value_lag_4', 'value_lag_5', 'Travel Agencies']]
    if label:
        y = df[label]
        return X, y
    return X

In [ ]:
df = df.replace(to_replace="<1",
           value=0.5)

In [ ]:
df_test = df_test.replace(to_replace="<1",
           value=0.5)

In [ ]:
def create_features(df, label=None):
    """
    Creates time series features from datetime index
    """
    df['date'] = df.index
    df['dayofweek'] = df['Date'].dt.dayofweek
    df['quarter'] = df['Date'].dt.quarter
    df['month'] = df['Date'].dt.month
    df['year'] = df['Date'].dt.year
    df['dayofyear'] = df['Date'].dt.dayofyear
    df['dayofmonth'] = df['Date'].dt.day
    df['weekofyear'] = df['Date'].dt.weekofyear

    X = df[['quarter','month','year','value_lag_1', 'value_lag_2', 'value_lag_3', 'value_lag_4', 'value_lag_5']]
    if label:
        y = df[label]
        return X, y
    return X

In [ ]:
df_test.columns.to_list()

['Date',
 'Rain',
 'T_max',
 'T_min',
 'Tourism',
 'Travel_index',
 'Travel',
 'Bus & Rail',
 'Tourist Destinations',
 'Travel Agencies',
 'value_lag_1',
 'value_lag_2',
 'value_lag_3',
 'value_lag_4',
 'value_lag_5',
 'Tourist Arrivals',
 'date',
 'dayofweek',
 'quarter',
 'month',
 'year',
 'dayofyear',
 'dayofmonth',
 'weekofyear']

In [ ]:
df_train.rename(columns = {'Kerala_Tourism':'Tourism'}, inplace = True)
df_train.rename(columns = {'Kerala_Travel':'Travel_index'}, inplace = True)

In [ ]:
split_date = '2020-01-05'
df_train = df.loc[df["Date"] <= split_date].copy()
df_test = df.loc[df["Date"] > split_date].copy()

In [ ]:
X_train, y_train = create_features(df_train, label='Rain')

# X, y = create_features(df, label='Rain')

<ipython-input-14-c2ddfe2599ec>:12: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['weekofyear'] = df['Date'].dt.weekofyear


In [ ]:
X_test = X_test[['value_lag_1', 'value_lag_2', 'value_lag_3', 'value_lag_4', 'value_lag_5'].copy()]

In [ ]:
X_test

,quarter,month,year,value_lag_1,value_lag_2,value_lag_3,value_lag_4,value_lag_5
120,1,1,2020,4.382383e+05,5.474205e+05,7.925275e+05,1.014022e+06,1.172773e+06
121,1,1,2020,3.603588e+05,4.382383e+05,5.474205e+05,7.925275e+05,1.014022e+06
122,1,1,2020,5.288536e+05,3.603588e+05,4.382383e+05,5.474205e+05,7.925275e+05
123,1,1,2020,6.207716e+05,5.288536e+05,3.603588e+05,4.382383e+05,5.474205e+05
124,1,1,2020,7.131644e+05,6.207716e+05,5.288536e+05,3.603588e+05,4.382383e+05
125,1,1,2020,8.403107e+05,7.131644e+05,6.207716e+05,5.288536e+05,3.603588e+05
126,1,1,2020,9.172054e+05,8.403107e+05,7.131644e+05,6.207716e+05,5.288536e+05
127,1,1,2021,1.173867e+06,9.172054e+05,8.403107e+05,7.131644e+05,6.207716e+05
128,1,1,2021,1.203317e+06,1.173867e+06,9.172054e+05,8.403107e+05,7.131644e+05
129,1,1,2021,1.088029e+06,1.203317e+06,1.173867e+06,9.172054e+05,8.403107e+05


In [ ]:
X_test, y_test = create_features(df_test, label='Rain')

<ipython-input-14-c2ddfe2599ec>:12: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['weekofyear'] = df['Date'].dt.weekofyear


In [ ]:
df_train.columns.to_list()

['Unnamed: 0',
 'Date',
 'Rain',
 'T_max',
 'T_min',
 'Tourism',
 'Travel_index',
 'Travel',
 'Bus & Rail',
 'Tourist Destinations',
 'Travel Agencies',
 'value_lag_1',
 'value_lag_2',
 'value_lag_3',
 'value_lag_4',
 'value_lag_5',
 'Cluster_ID',
 'Tourist Arrivals',
 'date',
 'dayofweek',
 'quarter',
 'month',
 'year',
 'dayofyear',
 'dayofmonth',
 'weekofyear']

In [ ]:
def calculate_rmse(actual_values, predicted_values):
    """
    Calculate Root Mean Square Error (RMSE) between actual and predicted values.

    Parameters:
    - actual_values: Array or list of actual values.
    - predicted_values: Array or list of predicted values.

    Returns:
    - rmse: Root Mean Square Error.
    """
    mse = mean_squared_error(actual_values, predicted_values)
    rmse = np.sqrt(mse)
    return rmse

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, accuracy_score

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.kernel_approximation import RBFSampler
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from math import sqrt
# X = X
# y = y

# # X['SKU id'] = X['SKU id'].replace({'SKU-': ''}, regex=True)
# # X['Warehouse id'] = X['Warehouse id'].replace({'Wh-': ''}, regex=True)
# #X.head()

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42, shuffle=False)
# Choose a kernel and set its parameters
gamma = 0.1
n_components = 100
rbf_feature = RBFSampler(gamma=gamma, n_components=n_components)

# Apply the kernel to the training data
x_train_transformed = rbf_feature.fit_transform(X_train)

# Train the Ridge Regression model using the transformed data
ridge_regressor = Ridge()
ridge_regressor.fit(x_train_transformed, y_train)

# Apply the same kernel to the test data
x_test_transformed = rbf_feature.transform(X_test)

# Make predictions on the test data
predictions = ridge_regressor.predict(x_test_transformed)

# Evaluate the regression performance
mse = mean_squared_error(y_test, predictions)
print(f"Mean Squared Error: {mse}")
mape = mean_absolute_percentage_error(y_test, predictions)

print(r2_score(y_test, predictions))
mape

Mean Squared Error: 206871453112.12378
-2.0177439698504016


0.3604863925550829

In [ ]:
import numpy as np
class ELM:
    def __init__(self, input_size, hidden_size, output_size, activation_function=None):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        # Randomly initialize input weights and biases
        self.input_weights = np.random.rand(hidden_size, input_size)
        self.biases = np.random.rand(hidden_size, 1)
        self.output_weights = None  # Will be set during training

        # Set the activation function
        if activation_function is None:
            self.activation_function = self.sigmoid
        else:
            self.activation_function = activation_function
        print("Activation function type:", type(self.activation_function))

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def fit(self, X, y):
        # Compute the hidden layer output
        hidden_output = self.activation_function(np.dot(self.input_weights, X.T) + self.biases)

        # Calculate output weights using the pseudoinverse
        self.output_weights = np.dot(np.linalg.pinv(hidden_output.T), y)

    def predict(self, X):
        # Compute hidden layer output for prediction
        hidden_output = self.activation_function(np.dot(self.input_weights, X.T) + self.biases)

        # Make predictions using output weights
        predictions = np.dot(hidden_output.T, self.output_weights)

        return predictions

    # The following methods are needed for compatibility with scikit-learn

    def get_params(self, deep=True):
        return {'input_size': self.input_size, 'hidden_size': self.hidden_size, 'output_size': self.output_size}

    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self, parameter, value)
        return self

# Example usage:
input_size = 100
hidden_size = 20
output_size = 10

# Create ELM model
elm_model = ELM(input_size, hidden_size, output_size)

# Generate random data for demonstration
# X_train = np.random.rand(100, input_size)
# y_train = np.random.rand(100, output_size)

# Train the ELM model
elm_model.fit(x_train_transformed, y_train)

# Make predictions on new data
# X_test = np.random.rand(10, input_size)
predictions = elm_model.predict(x_test_transformed)

Activation function type: <class 'method'>


In [ ]:
mape = mean_absolute_percentage_error(y_test, predictions)
print(calculate_rmse(y_test, predictions))
print(r2_score(y_test, predictions))
mape

457592.30846107565
-2.0544938676574924


0.35206665328232484

In [ ]:
X_train['Bus & Rail'] = y_train
X_test['Bus & Rail'] = predictions

In [ ]:
y_train = df_train['Tourist Destinations']
y_test = df_test['Tourist Destinations']

In [ ]:
# X_train = X_train.drop(['Traavel_index'], axis = 1)
# X_test = X_test.drop(['Travel_index'], axis = 1)

In [ ]:
X_train['Tourist Destinations'] = y_train
X_test['Tourist Destinations'] = predictions

In [ ]:
X_train

,quarter,month,year,value_lag_1,value_lag_2,value_lag_3,value_lag_4,value_lag_5,Rain,T_max,Travel_index,Travel,Bus & Rail,Tourist Destinations
0,1,1,2010,2.893800e+04,5.721500e+04,6.685400e+04,9.418500e+04,9.496600e+04,1.424957,0.665612,1.956571,1.948925,0.631997,2.130383
1,1,1,2010,2.673800e+04,2.893800e+04,5.721500e+04,6.685400e+04,9.418500e+04,1.925312,-0.359186,2.127279,2.119442,1.348376,1.638271
2,1,1,2010,4.093100e+04,2.673800e+04,2.893800e+04,5.721500e+04,6.685400e+04,0.622243,-0.895985,2.297986,2.289959,2.064754,2.212402
3,1,1,2010,5.623000e+04,4.093100e+04,2.673800e+04,2.893800e+04,5.721500e+04,0.068229,-0.950885,2.724755,2.716253,1.437923,3.114608
4,1,1,2010,4.467300e+04,5.623000e+04,4.093100e+04,2.673800e+04,2.893800e+04,0.639319,-0.944785,2.468694,2.460477,1.437923,2.212402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,1,1,2020,1.115650e+06,9.051488e+05,8.249393e+05,7.073838e+05,6.147513e+05,-0.892292,0.235867,-0.347981,-0.353058,-0.173928,-0.576234
116,1,1,2020,1.172773e+06,1.115650e+06,9.051488e+05,8.249393e+05,7.073838e+05,-0.892292,0.428932,-0.604042,-0.608834,-0.442570,-0.904309
117,1,1,2020,1.014022e+06,1.172773e+06,1.115650e+06,9.051488e+05,8.249393e+05,-0.610044,1.573900,-1.372226,-1.376162,-1.069401,-1.396421
118,1,1,2020,7.925275e+05,1.014022e+06,1.172773e+06,1.115650e+06,9.051488e+05,-0.193448,1.741222,-1.884349,-1.887714,-1.875327,-1.724496


In [ ]:
y_train = df_train['Travel Agencies']
y_test = df_test['Travel Agencies']

In [ ]:
X_train['Travel Agencies'] = y_train
X_test['Travel Agencies'] = predictions

In [ ]:
y_train = df_train['Tourist Arrivals']
y_test = df_test['Tourist Arrivals']

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

In [ ]:
elm_model = ELM(input_size, hidden_size, output_size)

# Define the parameter grid
param_grid = {
    'hidden_size': [10, 20, 30],
    'activation_function': ['sigmoid', 'tanh'],
    'regularization_strength': [0.1, 0.01, 0.001]
}

# Define your scoring metric (e.g., mean squared error for regression)
scoring = 'neg_mean_squared_error'

# Create the GridSearchCV object
grid_search = GridSearchCV(elm_model, param_grid, cv=5, scoring=scoring)

# Assuming you have a dataset X_train, y_train
grid_search.fit(X_train, y_train)

# Print the best parameters
print("Best parameters found: ", grid_search.best_params_)

# Get the best model
best_elm_model = grid_search.best_estimator_

# Make predictions on new data
predictions = best_elm_model.predict(X_test)

Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation 

ValueError: ignored

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# Create ELM model
elm_model = ELM(input_size, hidden_size, output_size)

# Define the parameter grid
param_grid = {
    'hidden_size': [10, 20, 30],
    'activation_function': ['sigmoid', 'tanh'],
}

# Define your scoring metric (e.g., mean squared error for regression)
scoring = 'neg_mean_squared_error'

# Create the GridSearchCV object
grid_search = GridSearchCV(elm_model, param_grid, cv=5, scoring=scoring)

# Assuming you have a dataset X_train, y_train
grid_search.fit(x_train_transformed, y_train)

# Print the best parameters
print("Best parameters found: ", grid_search.best_params_)

# Get the best model
best_elm_model = grid_search.best_estimator_

# Make predictions on new data
predictions = best_elm_model.predict(x_test_transformed)

Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation function type: <class 'method'>
Activation 

ValueError: ignored

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Generate synthetic regression data for demonstration
# X, y = make_regression(n_samples=1000, n_features=10, noise=0.1, random_state=42)

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the ELM model using MLPRegressor
elm_model = MLPRegressor(hidden_layer_sizes=(100,), activation='identity', solver='adam', random_state=42)

x_train_transformed = rbf_feature.fit_transform(X_train)


# Apply the same kernel to the test data
x_test_transformed = rbf_feature.transform(X_test)
# Train the ELM model
elm_model.fit(x_train_transformed, y_train)

# Make predictions on the test set
predictions = elm_model.predict(x_test_transformed)

# Evaluate the performance using Mean Squared Error
mse = mean_squared_error(y_test, predictions)
print(f"Mean Squared Error: {mse}")


Mean Squared Error: 2.257943164037837


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
mape = mean_absolute_percentage_error(y_test, predictions)
print(calculate_rmse(y_test, predictions))
print(r2_score(y_test, predictions))
mape

1.5026453886522386
-473.4485910519903


1.5511338213676829

In [ ]:
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (200,)],  # You can add more values
    'alpha': [0.0001, 0.001, 0.01]  # Regularization strength
}

# Create the ELM model (MLPRegressor)
elm_model = MLPRegressor(activation='identity', solver='adam', random_state=42)

# Create the GridSearchCV object
grid_search = GridSearchCV(elm_model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Perform hyperparameter tuning on the training data
grid_search.fit(x_train_transformed, y_train)

# Print the best parameters and their corresponding MSE
print("Best parameters found: ", grid_search.best_params_)
print("Best MSE on validation data: ", -grid_search.best_score_)

# Get the best model
best_elm_model = grid_search.best_estimator_

# Evaluate the best model on the test set
test_predictions = best_elm_model.predict(x_test_transformed)
test_mse = mean_squared_error(y_test, test_predictions)
print("Mean Squared Error on test data: ", test_mse)

Best parameters found:  {'alpha': 0.01, 'hidden_layer_sizes': (50,)}
Best MSE on validation data:  2.6623128644998726
Mean Squared Error on test data:  1.8692603941000134


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
mape = mean_absolute_percentage_error(y_test, test_predictions)
print(calculate_rmse(y_test, predictions))
print(r2_score(y_test, predictions))
mape

1.5026453886522386
-473.4485910519903


1.3957972784908104

In [ ]:
from sklearn.preprocessing import StandardScaler

# Scale the features
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train_transformed)
x_test_scaled = scaler.transform(x_test_transformed)

In [ ]:
from sklearn.pipeline import Pipeline
# Define the joint parameter space for both kernel and ELM
param_grid = {
    'kernel__gamma': [0.1, 1, 10],  # Adjust gamma for the RBF kernel
    'elm__hidden_layer_sizes': [(50,), (100,), (200,)],  # Hidden layer sizes
    'elm__alpha': [0.0001, 0.001, 0.01],  # Regularization strength for ELM
}

# Create the RBF kernel approximation
rbf_feature = RBFSampler(random_state=42)

# Create the ELM model (MLPRegressor) with RBF kernel
elm_model = MLPRegressor(activation='identity', solver='adam', random_state=42)

# Create a pipeline with the kernel and ELM
pipeline = Pipeline([
    ('kernel', rbf_feature),
    ('elm', elm_model)
])

# Create the GridSearchCV object
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Perform hyperparameter tuning on the training data
grid_search.fit(x_train_transformed, y_train)

# Print the best parameters and their corresponding MSE
print("Best parameters found: ", grid_search.best_params_)
print("Best MSE on validation data: ", -grid_search.best_score_)

# Get the best model
best_model = grid_search.best_estimator_

# Evaluate the best model on the test set
test_predictions = best_model.predict(x_test_transformed)
test_mse = mean_squared_error(y_test, test_predictions)
print("Mean Squared Error on test data: ", test_mse)

Best parameters found:  {'elm__alpha': 0.01, 'elm__hidden_layer_sizes': (50,), 'kernel__gamma': 0.1}
Best MSE on validation data:  1.4785226919707442
Mean Squared Error on test data:  0.895995380195327


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
mape = mean_absolute_percentage_error(y_test, test_predictions)
print(calculate_rmse(y_test, predictions))
print(r2_score(y_test, predictions))
mape

1.5026453886522386
-473.4485910519903


1.0098850702553275

In [ ]:
df_train.columns.to_list()

['Unnamed: 0',
 'Date',
 'Rain',
 'T_max',
 'T_min',
 'Tourism',
 'Travel_index',
 'Travel',
 'Bus & Rail',
 'Tourist Destinations',
 'Travel Agencies',
 'value_lag_1',
 'value_lag_2',
 'value_lag_3',
 'value_lag_4',
 'value_lag_5',
 'Cluster_ID',
 'Tourist Arrivals',
 'date',
 'dayofweek',
 'quarter',
 'month',
 'year',
 'dayofyear',
 'dayofmonth',
 'weekofyear']

In [ ]:
X_train['Travel Agencies'] = y_train
X_test['Travel Agencies'] = predictions
y_train = df_train['Travel Agencies']
y_test = df_test['Travel Agencies']

<ipython-input-240-f03a3c019796>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['Travel Agencies'] = y_train
<ipython-input-240-f03a3c019796>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['Travel Agencies'] = predictions


In [ ]:
X_train

,quarter,month,year,value_lag_1,value_lag_2,value_lag_3,value_lag_4,value_lag_5,Rain,T_max,T_min,Tourism
0,1,1,2010,2.893800e+04,5.721500e+04,6.685400e+04,9.418500e+04,9.496600e+04,1.424957,0.665612,0.701527,1.956571
1,1,1,2010,2.673800e+04,2.893800e+04,5.721500e+04,6.685400e+04,9.418500e+04,1.925312,-0.359186,0.348964,2.127279
2,1,1,2010,4.093100e+04,2.673800e+04,2.893800e+04,5.721500e+04,6.685400e+04,0.622243,-0.895985,0.569315,2.297986
3,1,1,2010,5.623000e+04,4.093100e+04,2.673800e+04,2.893800e+04,5.721500e+04,0.068229,-0.950885,0.619682,2.724755
4,1,1,2010,4.467300e+04,5.623000e+04,4.093100e+04,2.673800e+04,2.893800e+04,0.639319,-0.944785,0.355259,2.468694
...,...,...,...,...,...,...,...,...,...,...,...,...
115,1,1,2020,1.115650e+06,9.051488e+05,8.249393e+05,7.073838e+05,6.147513e+05,-0.892292,0.235867,-0.838984,-0.347981
116,1,1,2020,1.172773e+06,1.115650e+06,9.051488e+05,8.249393e+05,7.073838e+05,-0.892292,0.428932,-0.368376,-0.604042
117,1,1,2020,1.014022e+06,1.172773e+06,1.115650e+06,9.051488e+05,8.249393e+05,-0.610044,1.573900,0.738609,-1.372226
118,1,1,2020,7.925275e+05,1.014022e+06,1.172773e+06,1.115650e+06,9.051488e+05,-0.193448,1.741222,1.148841,-1.884349
